# ---------------------------------------------------------------------------------------------------------------


# Name - Malkan Kandarp M

# Roll : 202318021

# BDP : Kafka  Lab Assignment 

# ---------------------------------------------------------------------------------------------------------------


In [ ]:
#Consumer
from confluent_kafka import Consumer, KafkaError

def consume_inventory_data():
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'inventory_group'}

    consumer = Consumer(kafka_config)
    consumer.subscribe(['inventory_orders'])

    while True:
        msg = consumer.poll(timeout=1.0) 
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        inventory_data = json.loads(msg.value().decode('utf-8'))
        print("Received inventory data:", inventory_data)

def consume_delivery_data():
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'delivery_group'}

    consumer = Consumer(kafka_config)
    consumer.subscribe(['delivery_orders'])

    while True:
        msg = consumer.poll(timeout=1.0)  
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        delivery_data = json.loads(msg.value().decode('utf-8'))
        print("Received delivery data:", delivery_data)

consume_inventory_data()
consume_delivery_data()

In [ ]:
#Message Filter
import json

def filter_inventory_message(message):
    """
    Filter inventory messages based on the 'type' field.
    
    Args:
        message (str): JSON-encoded message.
        
    Returns:
        bool: True if the message type is 'inventory', otherwise False.
    """
    try:
        data = json.loads(message)
        return data.get('type') == 'inventory'
    except json.JSONDecodeError:
        return False

def filter_delivery_message(message):
    """
    Filter delivery messages based on the 'type' field.
    
    Args:
        message (str): JSON-encoded message.
        
    Returns:
        bool: True if the message type is 'delivery', otherwise False.
    """
    try:
        data = json.loads(message)
        return data.get('type') == 'delivery'
    except json.JSONDecodeError:
        return False

In [ ]:
#Producer
from confluent_kafka import Producer
import json

def produce_inventory_order():
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    producer = Producer(kafka_config)

    inventory_events = [
        {"type": "inventory", "item_id": "123", "quantity": 10},
        {"type": "inventory", "item_id": "456", "quantity": 20}
    ]

    for event in inventory_events:
        producer.produce('inventory_orders', json.dumps(event).encode('utf-8'))

    producer.flush()

def produce_delivery_order():
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    producer = Producer(kafka_config)

    delivery_events = [
        {"type": "delivery", "order_id": "1001", "status": "pending"},
        {"type": "delivery", "order_id": "1002", "status": "shipped"}
    ]

    for event in delivery_events:
        producer.produce('delivery_orders', json.dumps(event).encode('utf-8'))

    producer.flush()

produce_inventory_order()
produce_delivery_order()